## 実験的にコードを実行したい場合はこのファイルで

In [31]:
import argparse
import pathlib
import numpy as np
import sklearn.model_selection
from torch import utils
import torch
import torchvision
import torchvision.transforms.functional
from torchvision import transforms
import os

In [36]:
data_dir = './dogs-vs-cats-redux-kernels-edition'
train_path = os.path.join(data_dir, "train")

In [3]:
# mpsを利用できるか確認
if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")

else:
    print('mps is available')

mps is available


In [ ]:
def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--data_dir", required=True)
    parser.add_argument("--device", default="mps")
    args = parser.parse_args()

    data_dir = pathlib.Path(args.data_dir)

    print(data_dir)


if __name__ == "__main__":
    main()


In [5]:
def setup_center_crop_transform():
    return transforms.Compose(
        [
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]
    )

In [ ]:
def get_labels(dataset):
    if isinstance(dataset, torch.utils.data.Subset):
        return get_labels(dataset.dataset)[dataset.indices]
    else:
        return np.array([img[1] for img in dataset.imgs])
    

def setup_train_val_datasets(data_dir, dryrun=False):
    dataset = torchvision.datasets.ImageFolder(
        os.path.join(data_dir, "train"),
        transform=setup_center_crop_transform(),
    )
    labels = get_labels(dataset)
    return labels


    

In [8]:
dataset = torchvision.datasets.ImageFolder(
        os.path.join(data_dir, "train"),
        transform=setup_center_crop_transform(),
    )

if isinstance(dataset, torch.utils.data.Subset):
    get_labels(dataset.dataset)[dataset.indices]
else:
    np.array([img[1] for img in dataset.imgs])

In [9]:
len(np.array([img[1] for img in dataset.imgs]))

25000

In [6]:
torchvision.datasets.ImageFolder(
        os.path.join(data_dir, "train"),
        transform=setup_center_crop_transform()
    ).class_to_idx

{'cat': 0, 'dog': 1}

In [ ]:
for img in dataset.imgs:
    print(np.array(img[0]))

In [28]:
b = [1, 2, 3, 4, 5]
#listオブジェクトをイテレータオブジェクトに
b_iter = iter(b)
print(type(b_iter))
next(b_iter)

<class 'list_iterator'>


1

In [26]:
a = [1, 2, 3, 4, 5]
#listオブジェクトをイテレータオブジェクトに
iter_a = iter(a)
#nextメソッドで一つづつ取り出す
print(next(iter_a))
print(next(iter_a))

1
2


In [16]:
marks = [65, 71, 68, 74, 61]

# convert list to iterator
iterator_marks = iter(marks)

# the next element is the first element
marks_1 = next(iterator_marks)
print(marks_1)

# find the next element which is the second element
marks_2 = next(iterator_marks)
print(marks_2)

# Output: 65
#         71

65
71


In [11]:
l = ["みかん", "りんご", "バナナ", "パイナップル"]
it = iter(l)
while True:
    try:
        print(next(it))
    except StopIteration:
        break

みかん
りんご
バナナ
パイナップル


In [61]:
def setup_center_crop_transform():
    return transforms.Compose(
        [
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]
    )

def get_labels(dataset):
    if isinstance(dataset, torch.utils.data.Subset):
        return get_labels(dataset.dataset)[dataset.indices]
    else:
        return np.array([img[1] for img in dataset.imgs])
    


dataset = torchvision.datasets.ImageFolder(
    os.path.join(data_dir, "train"),
    transform=setup_center_crop_transform(),
)
labels = get_labels(dataset)


x = np.arange(len(labels))
y = np.array(labels)
#sklearn.model_selection.StratifiedShuffleSplit
#データセット全体のクラスの偏りを保持しながら、データセットを分割.データの重複が許容されていると分かる.必ずしも全てのデータがvalidationのデータセットに一度使われるわけではない.
#検証用データが重複したり，ある学習用データが学習されなかったりするので，あまり使われないイメージ
splitter = sklearn.model_selection.StratifiedShuffleSplit(
    n_splits=1, train_size=0.8, random_state=0    #n_splits:分割&シャッフル回数
)
train_indices, val_indices = next(splitter.split(x, y))
print(x)
print(y)
print(len(train_indices))
print(len(val_indices))
print(next(splitter.split(x, y)))
print(type(splitter.split(x, y)))

[    0     1     2 ... 24997 24998 24999]
[0 0 0 ... 1 1 1]
20000
5000
(array([15792, 17742,  8931, ..., 17930,  3981, 19080]), array([14589,  4039, 13467, ...,  9871, 21646, 12706]))
<class 'generator'>


In [46]:
labels

array([0, 0, 0, ..., 1, 1, 1])